# Trabajando con el DataSet sakila de MySQL 

In [2]:
import pandas as pd
import sqlite3 
import seaborn as sns
import matplotlib.pyplot as plt


db_file = 'sakila_master.db'


## Primer objetivo: ##
Analizar la relación entre la cantidad recaudada por las películas en la que aparece un actor, y la cantidad de películas entre la que pararece ese actor

In [3]:
# Esta consulta hace un recuento de la cantidad de apariciones en películas que tiene cada actor y la cantidad de ingresos totales que generó
query_1 = '''
-- hay 1000 películas, no obstante no hay información de los actores en 3 de ellas, sus IDs son 257, 323 y 803, no las tomaremos en cuenta 
WITH ts AS ( -- total_sales
SELECT 
	a.actor_id, 
	a.first_name || ' ' || a.last_name AS actor_name,
	sum(p.amount) AS total_sales
FROM actor a 

INNER JOIN film_actor fa ON a.actor_id = fa.actor_id 
INNER JOIN film f ON f.film_id = fa.film_id 
INNER JOIN inventory i ON i.film_id = f.film_id
INNER JOIN rental r ON r.inventory_id = i.inventory_id 
INNER JOIN payment p ON r.rental_id = p.rental_id 

GROUP BY a.actor_id 
), ta AS ( -- total_appearances
SELECT 
	a.actor_id, 
	a.first_name || ' ' || a.last_name AS actor_name,
	count(fa.film_id) AS appearances
FROM actor a 
INNER JOIN film_actor fa ON a.actor_id = fa.actor_id 
GROUP BY a.actor_id 
)

SELECT 
	ts.actor_id,
	ts.actor_name,
	ts.total_sales,
	ta.appearances
FROM ts
INNER JOIN ta ON ts.actor_id = ta.actor_id 
'''

In [4]:
with sqlite3.connect(db_file) as conn:
    df_sakila = pd.read_sql_query(query_1,conn)

df_sakila

,actor_id,actor_name,total_sales,appearances
0,1,PENELOPE GUINESS,1230.94,19
1,2,NICK WAHLBERG,1617.13,25
2,3,ED CHASE,1322.89,22
3,4,JENNIFER DAVIS,1052.27,22
4,5,JOHNNY LOLLOBRIGIDA,2027.04,29
...,...,...,...,...
195,196,BELA WALKEN,1917.50,30
196,197,REESE WEST,2299.47,33
197,198,MARY KEITEL,2689.25,40
198,199,JULIA FAWCETT,1189.42,15


In [ ]:
# existe una correlación directa, 0.87
df_sakila[['appearances', 'total_sales']].corr()


,appearances,total_sales
appearances,1.000000,0.867348
total_sales,0.867348,1.000000


Sabiendo esto, surje otra duda, serán los actores más activos, a su vez los más eficientes (más recaudación respecto a la cantidada de peliculas en las que participó) ? 

In [6]:
# Top 5 actores más eficientes 
df_sakila['actor_efficiency'] = df_sakila['total_sales'] / df_sakila['appearances'] 
((df_sakila.loc[:,['actor_name', 'actor_efficiency']]).sort_values(by = 'actor_efficiency', ascending=False)).iloc[0:5,:]

,actor_name,actor_efficiency
20,KIRSTEN PALTROW,86.832963
165,NICK DEGENERES,85.242273
150,GEOFFREY HESTON,84.168462
41,TOM MIRANDA,82.483333
162,CHRISTOPHER WEST,82.201429


In [7]:
# Sacamos el top 5 de los que más recaudaron
((df_sakila.loc[:,['actor_name', 'total_sales', 'actor_efficiency']]).sort_values(by = 'total_sales', ascending=False)).iloc[0:5,:]

,actor_name,total_sales,actor_efficiency
106,GINA DEGENERES,3442.49,81.964048
180,MATTHEW CARREY,2742.19,70.312564
197,MARY KEITEL,2689.25,67.231250
80,SCARLETT DAMON,2655.28,73.757778
101,WALTER TORN,2620.62,63.917561


In [8]:
# Sacamos el top 5 de los que más aparecieron 
((df_sakila.loc[:,['actor_name', 'appearances', 'actor_efficiency']]).sort_values(by = 'appearances', ascending=False)).iloc[0:5,:]

,actor_name,appearances,actor_efficiency
106,GINA DEGENERES,42,81.964048
101,WALTER TORN,41,63.917561
197,MARY KEITEL,40,67.231250
180,MATTHEW CARREY,39,70.312564
22,SANDRA KILMER,37,67.188108


In [9]:
# A simple vista no tiene nada que ver, saco la correlación para confirmar ]
df_sakila[['total_sales','appearances', 'actor_efficiency']].corr()

,total_sales,appearances,actor_efficiency
total_sales,1.000000,0.867348,0.500133
appearances,0.867348,1.000000,0.012553
actor_efficiency,0.500133,0.012553,1.000000


En principio la cantidad de apariciones no tiene nada que ver con la eficiencia del actor en cuestión, pero en contraste, el total de ventas si parece tener cierta relación con la eficiencia del actor, un resultado esperable teniendo en cuenta que la eficiencia del actor es determinada principalmente por lo que recauda en las películas en las que aparece

## Segundo objetivo: ##
Clasificar las películas, para hacerlo, voy a plantear el siguiente sistema:
- propensa a oferta (tercer nivel):
Tiene menos de 15 rentas: 
35% de descuento 

- Propensa a ofertas (segundo nivel):
tiene menos de 12 rentas y dura mas de 100 min:
35% de descuento + una película gratis de maximo 90 min 

- propensa a ofertas (primer nivel):
tiene menos de 10 rentas y en el cúmulo de actores de la película no hay ninguno popular, se toma en este caso como medida de popularidad de cada actor si el total_sales supera los $1837 que es el promedio de ventas de un actor:
50% de descuento + una película gratis 


In [10]:
query = """
SELECT
  i.film_id,
  f.length AS duration,
  r.rental_id,
  fa.actor_id,
  ActorSales.total_sales AS actor_total_sales
FROM inventory i

LEFT JOIN film f ON i.film_id = f.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
LEFT JOIN film_actor fa ON f.film_id = fa.film_id

LEFT JOIN (
  -- Subconsulta para obtener las ventas totales de cada actor
SELECT 
	a.actor_id, 
	sum(p.amount) AS total_sales
FROM actor a 

INNER JOIN film_actor fa ON a.actor_id = fa.actor_id 
INNER JOIN film f ON f.film_id = fa.film_id 
INNER JOIN inventory i ON i.film_id = f.film_id
INNER JOIN rental r ON r.inventory_id = i.inventory_id 
INNER JOIN payment p ON r.rental_id = p.rental_id 

GROUP BY a.actor_id 
  
) AS ActorSales ON fa.actor_id = ActorSales.actor_id
"""

# cargar el df
with sqlite3.connect(db_file) as conn:
    df_master = pd.read_sql_query(query, conn)


In [ ]:
# calculo con hiper groupby
df_agg = df_master.groupby('film_id').agg(
    amount_of_rent=('rental_id', 'nunique'),
    duration=('duration', 'first'),
    is_popular=('actor_total_sales', lambda x: (x > 1837).any())
)
df_agg

,amount_of_rent,duration,is_popular
film_id,,,
1,23,86,True
2,7,48,True
3,12,50,True
4,23,117,True
5,12,130,True
...,...,...,...
996,7,183,True
997,6,179,True
998,9,105,True


In [ ]:
# clasificador
def clasificador(row):
    cantidad_de_rentas = row['amount_of_rent']
    duracion = row['duration']
    es_popular = row['is_popular']
    if not es_popular and cantidad_de_rentas < 10:
        return 'Primer nivel'
    elif cantidad_de_rentas < 12 and duracion > 100:
        return 'Segundo nivel'
    elif cantidad_de_rentas < 15:
        return 'Tercer nivel'
    else:
        return 'Sin oferta'

df_agg['supply_level'] = df_agg.apply(clasificador, axis=1)
df_agg['supply_level'].value_counts()


supply_level
Sin oferta       580
Tercer nivel     224
Segundo nivel    143
Primer nivel      11
Name: count, dtype: int64

## Tercer objetivo: ##
Hacer un análisis de geográfico donde determinemos los paises, las ciudades, y las localidades de las cuales llegan más ingresos y por ende donde están los mejores clientes

In [13]:
# top 12 de los paises 
query_4 = '''
SELECT 
	c.country_id,
	c.country,
	sum (p.amount) AS total_collected 
FROM country c 
INNER JOIN city ct ON c.country_id = ct.country_id 
INNER JOIN address a ON a.city_id = ct.city_id 
INNER JOIN customer cus ON cus.address_id = a.address_id 
INNER JOIN payment p ON cus.customer_id = p.customer_id 
GROUP BY c.country_id
ORDER BY total_collected DESC 
LIMIT 12 
'''

# top 20 de las ciudades (en este caso un top más amplio por la diferencia de volumen de los datos)
query_5 = '''
SELECT 
	ct.city_id,
	ct.city,
	c.country,
	sum (p.amount) AS total_collected
FROM country c 
INNER JOIN city ct ON c.country_id = ct.country_id 
INNER JOIN address a ON a.city_id = ct.city_id 
INNER JOIN customer cus ON cus.address_id = a.address_id 
INNER JOIN payment p ON cus.customer_id = p.customer_id 
GROUP BY ct.city_id
ORDER BY total_collected DESC 
LIMIT 20
'''

# en este caso, en la base de datos solo hay aproximadamente un código postal por ciudad, así que no es muy relevante tomarlo en cuenta, ya que sería
# practicamente lo mismo de las ciudades 
query_6 = '''
SELECT 
	a.address_id,
	a.address,
	ct.city,
	c.country,
	sum (p.amount) AS total_collected
FROM country c 
INNER JOIN city ct ON c.country_id = ct.country_id 
INNER JOIN address a ON a.city_id = ct.city_id 
INNER JOIN customer cus ON cus.address_id = a.address_id 
INNER JOIN payment p ON cus.customer_id = p.customer_id 
GROUP BY a.address_id
ORDER BY total_collected DESC 
'''

In [14]:

with sqlite3.connect(db_file) as conn:
    df_paises = pd.read_sql_query(query_4,conn)
    df_ciudades = pd.read_sql_query(query_5,conn)

In [15]:
df_paises

,country_id,country,total_collected
0,44,India,6630.27
1,23,China,5802.73
2,103,United States,4110.32
3,50,Japan,3471.74
4,60,Mexico,3307.04
5,15,Brazil,3200.52
6,80,Russian Federation,3045.87
7,75,Philippines,2381.32
8,97,Turkey,1662.12
9,69,Nigeria,1511.48


In [16]:
df_ciudades

,city_id,city,country,total_collected
0,101,Cape Coral,United States,221.55
1,442,Saint-Denis,Runion,216.54
2,42,Aurora,United States,198.50
3,340,Molodetno,Belarus,195.58
4,456,Santa Brbara dOeste,Brazil,194.61
5,29,Apeldoorn,Netherlands,194.61
6,423,Qomsheh,Iran,186.62
7,312,London,United Kingdom,180.52
8,388,Ourense (Orense),Spain,177.60
9,78,Bijapur,India,175.61
